<a href="https://colab.research.google.com/github/mwaoh/Twitter-Sentiment-Analysis/blob/main/Twitter_Word_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#Getting packages and libraries to use

In [2]:
import io
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import TextVectorization

In [4]:
!pip3 install snscrape
# Importing necessary libraries
import snscrape.modules.twitter as sntwitter
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 KB 8.6 MB/s eta 0:00:00


#Data Preprocessing for Sentiment Analysis and Word Cloud



In [5]:
!pip3 install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
def search_hashtag(searchterm, dt_until, dt_since, lang, limit=100000):
    query = "({searchterm}) lang:{lang} until:{until} since:{since}".format(searchterm=searchterm, lang=lang, until=dt_until, since=dt_since)
    #query = (#Emmys2022) lang:en until:2022-09-14 since:2022-09-01
    
    tweets = []
    limit = limit
    
    q = sntwitter.TwitterSearchScraper(query)
    for tweet in q.get_items():
        if len(tweets) == limit:
            break
        else:
            tweets.append([tweet.date, tweet.content])
    
    df = pd.DataFrame(tweets, columns=['Date', 'Tweet'])
    
    csv_name = "data/Emmys2022/" + searchterm.replace("#","hastag_").replace("\"","").replace(" ","_") + "_" + str(since) + "_" + str(until)  + ".csv"
    
    df.to_csv(csv_name, index=False)
    
    print("succecss")

In [12]:
from nltk.corpus import stopwords
nltk.download()
nltk.download('stopwords')
print(stopwords.words('english'))

NameError: ignored